## Introduction 
The **World Factbook** provides basic intelligence on the history, people, government, economy, energy, geography, environment, communications, transportation, military, terrorism, and transnational issues for 266 world entities.

In this project, we'll work with data from the CIA World Factbook found [here](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth.


*Connect Jupyter Notebook to our database file*

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql 
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Overview

Here are the descriptions for some of the columns:
<ul>
<li> **name** - The name of the country.</li>
<li> **area** - The total land and sea area of the country.</li>
<li> **population** - The country's population.</li>
<li> **population_growth** - The country's population growth as a percentage</li>
<li> **birth_rate** - The country's birth rate, or the number of births a year per 1,000 people.</li>
<li> **death_rate** - The country's death rate, or the number of death a year 1,000 people.</li>
<li> **area** - The country's total area (both land and water).</li>
<li> **area_land** - The country's land area in sq. km</li>
<li> **area_water** - The country's waterarea in sq. km </li>
</ul>

## Calculating some summary stats 

In [4]:
%%sql
SELECT MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


##  <span style='color:blue'> Weird numbers </span>

There's a country with a population of 0
There's a country with a population of 7256490011 (or more than 7.2 billion people aka world's population)


##  <span style='color:red'> Exploring Outliers </span>
Find the countries with the minimum and maximum popluation

In [20]:
%%sql
SELECT *
FROM facts
WHERE population = 0

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Observation
It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html)

Antarctica is known for being the highest, driest, coldest and windiest continent on earth. Although there are no native Antarcticans and no permanent residents OF Antarctica, many people do live in Antarctica each year. So, since there are no permanent residents, then on that basis the populaiton is deemed to be 0.

## Summary Statistics Revisited
By excluding World

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
FROM facts
WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [8]:
%%sql
SELECT name AS Country, population AS Population
  FROM facts
ORDER BY population DESC
LIMIT 7;

Done.


Country,Population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812


We see that the most populated country is China, followed by India, then with a big gap by the United States, Indonesia, and Brazil.
European Union not a country 

In [9]:
%%sql
SELECT name, population
  FROM facts
WHERE population IS NULL;

Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None



All these territories represent or different uninhabited islands, or the oceans.

In [10]:
%%sql
SELECT name AS Country, population AS Population
  FROM facts
WHERE population NOT NULL AND name <> 'Antarctica'
ORDER BY population 
LIMIT 5;

Done.


Country,Population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337


For the BOTTOM5 countries, everything is coherent. The Pitcairn Islands, or just Pitcairn, located in the southern Pacific Ocean, is the least populous national jurisdiction in the world. Niue and Tokelau are also very isolated islands in the southern Pacific Ocean, while Cocos Islands – in the Indian Ocean. Among the BOTTOM5 countries by population, not surprisingly, there is also a microstate Vatican.

## Observations : Minimum  and Maximum Population 
We now have a number greater than 0 for minimum (48) which is  Pitcairn Islands and a number less than the world population for maximum (1,367,485,388) 
that is China.

## Finding Densely Populated Coutries

In [11]:
%%sql
SELECT AVG(Population), AVG(area)
FROM facts
WHERE name <> 'World'

Done.


AVG(Population),AVG(area)
32242666.56846473,555093.546184739


**Criteria** :
<ul>
<li>Population above average</li>
<li>Area below average </li>
</ul>

In [12]:
%%sql
SELECT name 
FROM facts
WHERE population > (SELECT AVG(Population)
                    FROM facts
                    WHERE name <> 'World') 
AND area < (SELECT AVG(area)
            FROM facts
            WHERE name <> 'World')
ORDER BY population/area 


Done.


name
Morocco
Iraq
Spain
Poland
Thailand
Uganda
Italy
Germany
United Kingdom
Vietnam


## Observation : Population Density 
#### area_land would be a better indicator 
<ul>
<li>use area_land instead of area, for being more precise </li>
<li>exclude null values for both population and area_land </li>
<li>exclude area_land=0 to avoid having 0 in the denominator </li>
<li>exclude the outliers: World, European Union, and Antarctica </li>
</ul>

In [13]:
%%sql
SELECT *
  FROM facts
WHERE name IN ('World','European Union','Antarctica');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


For avg population, world and european union have to be excluded 
for avg area_land , EU and World has none listed 

In [14]:
%%sql
SELECT name, CAST(population AS FLOAT)/CAST(area_land AS FLOAT) as density
FROM facts
WHERE (population > (SELECT AVG(Population)
                    FROM facts
                    WHERE name <> 'World') 
AND area_land < (SELECT AVG(area_land)
                FROM facts
                WHERE (name NOT IN ('World' ,'European Union')
                AND area_land <> 0)))
ORDER BY density DESC;



Done.


name,density
Bangladesh,1297.9776062072674
"Korea, South",506.7601733388361
Japan,348.21641219803286
Philippines,338.72749102860786
Vietnam,304.28237172251426
United Kingdom,264.9039887570785
Germany,231.8924605359765
Italy,210.29142585163527
Uganda,188.23817858954845
Thailand,133.05487482628354


### Observation regarding the results: 
According to [Statista](https://www.statista.com/statistics/264683/top-fifty-countries-with-the-highest-population-density/), some of the countries do match but Macau , Monaco , China , Singapore are not included in the list

In [15]:
%%sql
SELECT name ,population , area_land , area , population/area_land as density
FROM facts
WHERE name IN ('Monaco','Singapore','Bahrain');

Done.


name,population,area_land,area,density
Bahrain,1346613,760,760,1771
Monaco,30535,2,2,15267
Singapore,5674472,687,697,8259


In [16]:
%%sql
SELECT name, population, area_land, 
       ROUND(CAST(population AS Float)/CAST(area_land AS Float), 0) AS population_density
FROM facts
WHERE population_density >(SELECT CAST(AVG(population) AS Float)/CAST(AVG(area_land) AS Float) 
                          FROM facts
                          WHERE name != 'Antarctica'
                          AND population NOT NULL 
                          AND area_land NOT NULL
                          AND area_land != 0)
ORDER BY population_density DESC
LIMIT 20;

Done.


name,population,area_land,population_density
Macau,592731,28,21169.0
Monaco,30535,2,15268.0
Singapore,5674472,687,8260.0
Hong Kong,7141106,1073,6655.0
Gaza Strip,1869055,360,5192.0
Gibraltar,29258,6,4876.0
Bahrain,1346613,760,1772.0
Maldives,393253,298,1320.0
Malta,413965,316,1310.0
Bermuda,70196,54,1300.0


### <span style='color:green'> Observations - Densely populated </span>
According to this table, the highest population density is observed in Macau, followed with a big gap by Monaco, which in turn is followed with another big gap by Singapore and Hong Kong. In general, as we can see, these high values are mostly related to small countries and islands, with the area much below average  km2), but also the population below average 

#### Which country has the highest growth rate?

## Exploring Population Growth Statistics

Population growth rate compares the average annual percent change in populations, resulting from a surplus (or deficit) of births over deaths and the balance of migrants entering and leaving a country. The rate may be positive or negative.

In [17]:
%%sql
SELECT name, population_growth 
FROM facts    
ORDER BY population_growth DESC
LIMIT 20;



Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


### Observations
The population growth rate in African countries has been the highest and also in Qatar ,Iraq and Cook Islands.


Factors like political and economic instability, high crime rates and war, higher emigration to immigration ratio, low birth rates, low fertility rates, racial persecutions, poverty and unemployment, natural disasters, and severe weather conditions can cause a country to experience a negative population growth rate

In [18]:
%%sql
SELECT *
FROM facts    
WHERE population_growth = 0 OR population_growth < 0;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


In [19]:
%%sql
SELECT * , ROUND(CAST(area_water AS float)/area_land,4) as water_land_ratio
FROM facts   
WHERE (area_water != 'None') AND (area_water != 0) AND (area_land != 'None') AND (area_land != 0)
ORDER BY water_land_ratio DESC
LIMIT 20;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6667
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.5202
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5548
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2847
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.2594
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.2257
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.2229
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.1644
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.1562


There are only two "countries" shown in the table above identified as having greater water area than land area: The British Indian Ocean Territory and the Virgin Islands. 



The British Indian Ocean Territory (BIOT) shown in map below, enjoys a highly strategic location in the centre of the Indian Ocean, about 1,770 kilometres east of the Seychelles and 2,180km north-east of Port Louis, Mauritius. The territory, an archipelago of 58 islands also known as the Chagos Archipelago, covers some 640,000km2 of ocean. The islands have a total land area of only 60km2. Diego Garcia, the largest and most southerly island, has an area of 44 square km. In 1971, Diego Garcia was closed when the island was deemed necessary for defence purposes under the 1966 UK-US Exchange of Notes. Diego Garcia is home to approximately 3,000 UK and US military personnel and civilian contractors.

Virgin Islands - Of the 36 British islands, 16 are inhabited. Tortola (Turtle Dove), with an area of 21 square miles, is the largest and is the site of the group’s capital and population centre, Road Town. Other larger islands in the British group are Anegada, with an area of 15 square miles; Virgin Gorda (the Fat Virgin), with an area of 8 square miles; and Jost Van Dyke, about 3 square miles. Lesser islands include Great Tobago, Salt, Peter, Cooper, Norman, Guana, Beef, Great Thatch, Little Thatch, and Marina Cay.

Puerto Rico - Even though the ratio of water area to land area for Puerto Rico is less than 1 (0.5548), I believe that value is also not legitimate for the same reason as the two cases above (see map of Puerto Rico further below). I read the following about Puerto Rico: "Although, often referred an island, Puerto Rico is an archipelago formed by the main island and 143 small islands, islets and cays, including Vieques, Culebra, Culebrita, Palomino, Mona, and Monito. Vieques and Culebra are also known by some by the Spanish Virgin Islands, formerly called the Passage Islands and also known as the Puerto Rican Virgin Islands."

I would say the ratio values shown above for The Bahamas and Guinea-Bassau are also inaccurate for the same reason because of the inclusion of many islands.

It looks like the country with the highest legitimate water area to land area ratio is Malawi (0.2847). We would say that about 25% of Malawi is covered with water. The map further below makes that value look reasonable. Malawi, a landlocked country in southeastern Africa, is defined by its topography of highlands split by the Great Rift Valley and enormous Lake Malawi. The lake’s southern end falls within Lake Malawi National Park – sheltering diverse wildlife from colorful fish to baboons – and its clear waters are popular for diving and boating. Peninsular Cape Maclear is known for its beach resorts.

In conclusion, I would say that no country has more than 30% of its land covered with water.